In [ ]:
import librosa
import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import numpy as np

from torch.utils.data import DataLoader, Dataset

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [ ]:
!nvidia-smi

Wed Jul 27 14:31:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
!pip install -U pip gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=6929e953daab53a8518a0af576701a3b352d39e96c83f115fb32207744ab3d67
  Stored in directory: /root/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
# https://drive.google.com/file/d/1eLVEr6dvycpzCBsW7qcpYE5LoSahgz_W/view?usp=sharing
import gdown, os
os.chdir('/content')
id = '1eLVEr6dvycpzCBsW7qcpYE5LoSahgz_W'
url = "https://drive.google.com/uc?id={}".format(id)
output = "VCC2018_MOS_preprocessed.tar.bz"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1eLVEr6dvycpzCBsW7qcpYE5LoSahgz_W
To: /content/VCC2018_MOS_preprocessed.tar.bz
100%|██████████| 1.32G/1.32G [00:10<00:00, 123MB/s]


'VCC2018_MOS_preprocessed.tar.bz'

In [ ]:
import tarfile

tar = tarfile.open(output, "r:bz2")
tar.extractall()
tar.close()

In [ ]:
import pandas as pd

df = pd.read_csv('/content/VCC2018_MOS_preprocessed/mos_list.txt', header=None)
df[0] = '/content/VCC2018_MOS_preprocessed/wav/' + df[0] 
df.columns = ['filepath', 'score']
df.head()

,filepath,score
0,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,1.25
1,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,2.75
2,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,3.50
3,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,1.75
4,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,4.00


In [ ]:
num_classes = len(df['score'].unique())
num_classes

21

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df['score'])
df['score'] = le.transform(df['score'])

In [ ]:
from tqdm import tqdm
import torchaudio
data = []

wavs_folder = '/content/VCC2018_MOS_preprocessed/wav/'
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    #filepath = os.path.join(wavs_folder, row['path'])
    filepath = row['filepath']
    score = row['score']
    try:
        # There are some broken files
        s = torchaudio.load(filepath)
        data.append({
            # "name": name,
            "filepath": filepath,
            "score": score
        })
    except Exception as e:
        #print(str(filepath), e)
        pass

100%|██████████| 22360/22360 [00:13<00:00, 1604.36it/s]


In [ ]:
data = pd.DataFrame(data)

In [ ]:
data

,filepath,score
0,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,1
1,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,7
2,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,11
3,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,3
4,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,14
...,...,...
20575,/content/VCC2018_MOS_preprocessed/wav/N14_VCC2...,6
20576,/content/VCC2018_MOS_preprocessed/wav/N10_VCC2...,17
20577,/content/VCC2018_MOS_preprocessed/wav/N09_VCC2...,3
20578,/content/VCC2018_MOS_preprocessed/wav/N11_VCC2...,7


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.10) 

In [ ]:
categories_train = train_data['score'].to_list()
audio_names_train = train_data['filepath'].to_list()

categories_test = test_data['score'].to_list()
audio_names_test = test_data['filepath'].to_list()

In [ ]:
# get all unique categories
categories_types = np.sort(df['score'].unique())
categories_types, categories_types.shape

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20]), (21,))

In [ ]:
# if we don't do preprocessing - saving specs - use this dataset 
class VCC2018MOSSpecDatasetLoad(Dataset):
    def __init__(self, audio_names: list, labels: list):
        self.audio_names = audio_names
        self.labels = labels
        self.transformation = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,
            n_fft=1024,
            hop_length=512,
            n_mels=80
        )
        #self.label_to_id = dict((mos,id) for id, mos in enumerate(labels))
        
    def __len__(self):
        return len(self.audio_names)

    def __getitem__(self, idx):
        filename = self.audio_names[idx]
        #waveform, sample_rate = torchaudio.load(filename)
        wav, sr = torchaudio.load(filename)
        mel_spec = self.transformation(wav).unsqueeze(1)

        target = self.labels[idx]
        
        return {"spec": mel_spec, "target": target}

    def get_transform_spec(self, idx, data_dir='./'):
        # get audio path
        specs = []
        audio_name = self.audio_names[idx]
        audio_path = os.path.join(data_dir, audio_name)
          
        # load audio and get its melspectrogram
        audio_wave, sr = librosa.load(audio_path, sr=None)

        mel_spec = librosa.feature.melspectrogram(y=audio_wave, sr=sr)
        
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec = torch.Tensor(self.normalize(mel_spec)).unsqueeze(0)
        return mel_spec        

In [ ]:
from torch.nn.utils.rnn import pad_sequence 

def wav_collate_fn(data):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    features = [torch.tensor(d['spec']) for d in data] #(3)
    labels = torch.tensor([d['target']  for d in data]) 
    new_features = pad_sequence([f.T for f in features], batch_first=True).squeeze()

    return  {
        'spec': new_features.to(device), 
        'target': labels.to(device)
    }

In [ ]:
dataset_train = VCC2018MOSSpecDatasetLoad(audio_names_train, categories_train)
loader_train = DataLoader(dataset_train, batch_size=96, shuffle=True, collate_fn=wav_collate_fn)

dataset_test = VCC2018MOSSpecDatasetLoad(audio_names_test, categories_test)
loader_test = DataLoader(dataset_test, batch_size=8, shuffle=True, collate_fn=wav_collate_fn)


In [ ]:
for i, batch in enumerate(loader_train):
    print(type(batch['spec']))
    print(batch['spec'].shape)
    print(batch['spec'].get_device())
    print(type(batch['target']))
    print(batch['target'].shape)    
    print(batch['target'].get_device())
    break
    

<class 'torch.Tensor'>
torch.Size([96, 241, 80])
0
<class 'torch.Tensor'>
torch.Size([96])
0


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

        self._init_weights()
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.zeros_(m.bias)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.avg_pool2d(x, 2)
        return x

In [ ]:
class Classifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.conv = nn.Sequential(
            ConvBlock(in_channels=1, out_channels=64),
            ConvBlock(in_channels=64, out_channels=128),
            ConvBlock(in_channels=128, out_channels=256),
            ConvBlock(in_channels=256, out_channels=512),
        )
        
        self.fc = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.PReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.conv(x)
        x = torch.mean(x, dim=3)
        x, _ = torch.max(x, dim=2)
        x = self.fc(x)
        return x

In [ ]:
model = Classifier(num_classes=num_classes)

In [ ]:
x = torch.rand(3, 180, 80).permute(0,2,1).unsqueeze(1)
x.shape

torch.Size([3, 1, 80, 180])

In [ ]:
y = model(x)
y.shape

torch.Size([3, 21])

Training

In [ ]:
import time
import math
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def train(model, iterator, optimizer, criterion, scheduler, epoch=0):
    model.train()
    
    epoch_loss, accuracy, f1, recall, precision = 0, 0, 0, 0, 0 

    total_steps = len(iterator)
    for i, batch in enumerate(iterator):
        specs = batch['spec'].to(device, dtype=torch.float32).unsqueeze(1)
        labels = batch['target'].to(device)

        optimizer.zero_grad()
        
        output = model(specs)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
        #calculate metrics
        output = F.softmax(output, 1)
        result = output.argmax(1)
            
        accuracy += accuracy_score(result.cpu(), labels.cpu())
        f1 += f1_score(result.cpu(), labels.cpu(), average='micro')
        recall += recall_score(result.cpu(), labels.cpu(), average='micro')
        precision += precision_score(result.cpu(), labels.cpu(), average='micro')

        if (i % 10 == 0):
          step_loss = "{:.5f}".format(epoch_loss / (i+1))
          step_acc = "{:.5f}".format(accuracy / (i+1))
          step_f1 = "{:.5f}".format(f1 / (i+1))
          step_recall = "{:.5f}".format(recall / (i+1))
          step_precision = "{:.5f}".format(precision / (i+1))
          print(f"Train step {i} loss: {step_loss} acc: {step_acc} f1: {step_f1} recall {step_recall} precision: {step_precision}")
    #wandb.log({"loss_train": epoch_loss / (i+1), "accuracy_train": accuracy / (i+1), "f1_train": f1 / (i+1), "recall_train": recall / (i+1), "precision_train": precision / (i+1)})

    accuracy /= (i+1)
    f1 /= (i+1)
    epoch_loss /= (i+1)
    recall /= (i+1)
    precision /= (i+1)

    return epoch_loss, accuracy, f1, recall, precision

def evaluate(model, iterator, criterion, epoch):
    model.eval()
    epoch_loss, accuracy, f1, recall, precision = 0, 0, 0, 0, 0 
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            specs = batch['spec'].to(device, dtype=torch.float32).unsqueeze(1)
            batch_size = specs.shape[0]
            
            labels = batch['target'].to(device)
            output = model(specs)
            loss = criterion(output, labels)
            
            epoch_loss += loss.item()
            result = output.argmax(1)
            
            accuracy += accuracy_score(result.cpu(), labels.cpu())
            f1 += f1_score(result.cpu(), labels.cpu(), average='micro')
            recall += recall_score(result.cpu(), labels.cpu(), average='micro')
            precision += precision_score(result.cpu(), labels.cpu(), average='micro')
        
    #wandb.log({"loss_train": epoch_loss / (i+1), "accuracy_train": accuracy / (i+1), "f1_train": f1 / (i+1), "recall_train": recall / (i+1), "precision_train": precision / (i+1)})

    accuracy /= (i+1)
    f1 /= (i+1)
    epoch_loss /= (i+1)
    recall /= (i+1)
    precision /= (i+1)
    
    return epoch_loss, accuracy, f1, recall, precision

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# wandb.init(project='emble_audio_classification', entity='miana')
# config = wandb.config

# change model below
# model = CNNNetwork().to(device)
model = Classifier(num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-1)
criterion = nn.CrossEntropyLoss()

lambda2 = lambda epoch: epoch * 0.95
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer,  lr_lambda=[lambda2])

In [ ]:
  N_EPOCHS = 10

  train_losses, val_losses = [], []
  train_accuracies, val_accuracies = [], []
  train_f1s, val_f1s = [], []

  best_valid_loss = float('inf')

  #wandb.watch(model)
  for epoch in range(N_EPOCHS):
      start_time = time.time()
      train_loss, train_accuracy, train_f1, train_recall, train_precision = train(model, loader_train, optimizer, criterion, scheduler, epoch)
      val_loss, val_accuracy, val_f1, val_recall, val_precision = evaluate(model, loader_test, criterion, epoch)
      
      end_time = time.time()
      
      # fill data
      train_losses.append(train_loss)
      val_losses.append(val_loss)
      train_accuracies.append(train_accuracy)
      train_f1s.append(train_f1)
      val_accuracies.append(val_accuracy)
      val_f1s.append(val_f1)
      
      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      torch.save(model.state_dict(), 'best-val-model.pt')

      print(f'Epoch: {epoch+1:02} | Time: {epoch_mins} m {epoch_secs} s')
      print(f'\tTrain Loss: {train_loss}, accuracy: {train_accuracy}, f1 {train_f1}, recall {train_recall}, precision {train_precision}')
      print(f'\t Val. Loss: {val_loss}, accuracy: {val_accuracy}, f1 {val_f1}, recall {val_recall}, precision {val_precision}')

Train step 0 loss: 3.14848 acc: 0.04167 f1: 0.04167 recall 0.04167 precision: 0.04167
Train step 10 loss: 3.18255 acc: 0.05303 f1: 0.05303 recall 0.05303 precision: 0.05303
Train step 20 loss: 3.18272 acc: 0.05109 f1: 0.05109 recall 0.05109 precision: 0.05109
Train step 30 loss: 3.18904 acc: 0.04973 f1: 0.04973 recall 0.04973 precision: 0.04973
Train step 40 loss: 3.18991 acc: 0.04802 f1: 0.04802 recall 0.04802 precision: 0.04802
Train step 50 loss: 3.18614 acc: 0.04841 f1: 0.04841 recall 0.04841 precision: 0.04841
Train step 60 loss: 3.18214 acc: 0.04764 f1: 0.04764 recall 0.04764 precision: 0.04764
Train step 70 loss: 3.18971 acc: 0.04739 f1: 0.04739 recall 0.04739 precision: 0.04739
Train step 80 loss: 3.19204 acc: 0.04771 f1: 0.04771 recall 0.04771 precision: 0.04771
Train step 90 loss: 3.19120 acc: 0.04796 f1: 0.04796 recall 0.04796 precision: 0.04796
Train step 100 loss: 3.19025 acc: 0.04755 f1: 0.04755 recall 0.04755 precision: 0.04755
Train step 110 loss: 3.19192 acc: 0.04683 f